In [ ]:
# stdlib
import os
from pathlib import Path

# syft absolute
import syft as sy

In [ ]:
server_low = sy.orchestra.launch(
    name="bigquery-low",
    server_side_type="low",
    dev_mode=True,
    n_consumers=1,
    create_producer=True,
    port="auto",
)

server_high = sy.orchestra.launch(
    name="bigquery-high",
    server_side_type="high",
    dev_mode=True,
    n_consumers=1,
    create_producer=True,
    port="auto",
)

In [ ]:
low_client = server_low.login(email="info@openmined.org", password="changethis")
high_client = server_high.login(email="info@openmined.org", password="changethis")

# Dump low side

In [ ]:
migration_data_dir = Path(os.getenv("MIGRATION_DATA_DIR", "."))
migration_data_dir.mkdir(exist_ok=True)

low_blob_path = migration_data_dir / "migration_low.blob"
low_yaml_path = migration_data_dir / "migration_low.yaml"

low_blob_path.unlink(missing_ok=True)
low_yaml_path.unlink(missing_ok=True)

In [ ]:
# Dump low state

low_migration_data = low_client.get_migration_data(include_blobs=True)
low_migration_data

In [ ]:
low_migration_data.save(path=low_blob_path, yaml_path=low_yaml_path)

# Dump high side

In [ ]:
high_blob_path = migration_data_dir / "migration_high.blob"
high_yaml_path = migration_data_dir / "migration_high.yaml"

high_blob_path.unlink(missing_ok=True)
high_yaml_path.unlink(missing_ok=True)

In [ ]:
high_migration_data = high_client.get_migration_data(include_blobs=True)
high_migration_data

In [ ]:
high_migration_data.save(path=high_blob_path, yaml_path=high_yaml_path)

In [ ]:
for path in (low_blob_path, low_yaml_path, high_blob_path, high_yaml_path):
    assert path.exists(), f"Migration file {path} does not exist"

In [ ]:
if server_high.server_type.value == "python":
    server_high.land()

if server_low.server_type.value == "python":
    server_low.land()